# Prediction system

## This is a python notebook where you can upload your sit-to-stand data and get prediction results!

In [11]:
# import required libraries
import pickle
import pandas as pd
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

# define turn 2d features
features = [f'x{i}' for i in range(0, 16)] + [f'y{i}' for i in range(0, 16)] + ['DBS_state']

# define the long short-term memory classifier
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=2, num_layers=3):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# import the scaler fitted ot the model
scaler_file_path = '/Users/suhrudp/Library/CloudStorage/OneDrive-Personal/Stats/REMAP Open Dataset PD/Analysis/Models/turn2d_scaler.pkl'
with open(scaler_file_path, 'rb') as file:
    scaler = pickle.load(file)

# import the lstm model weights
model_file_path = '/Users/suhrudp/Library/CloudStorage/OneDrive-Personal/Stats/REMAP Open Dataset PD/Analysis/Models/turn2d_model_weights.pth'
model = LSTMClassifier(input_dim=35, hidden_dim=120, num_layers=3)
model.load_state_dict(torch.load(model_file_path))
model.eval()

def preprocess_new_data(filepath, scaler):
    new_data = pd.read_csv(filepath, header=None, skiprows=1)
    column_names = [f'x{i}' for i in range(0, 17)] + [f'y{i}' for i in range(0, 17)]
    new_data.columns = column_names
    new_data['DBS_state'] = 1
    features = [col for col in column_names[0:] + ['DBS_state'] if col in new_data.columns]
    new_data[features] = scaler.transform(new_data[features])
    return new_data[features]

# define the model prediction function
def predict(model, new_data):
    with torch.no_grad():
        new_data = torch.tensor(new_data.to_numpy(dtype=np.float32))
        
        if new_data.ndim == 2:
            new_data = new_data.unsqueeze(0)
        
        outputs = model(new_data)
        _, predicted = torch.max(outputs.data, 1)
        
        return ["PD" if label == 1 else "Control" for label in predicted]

# prediction using the model
filepath = '/Users/suhrudp/Library/CloudStorage/OneDrive-Personal/Stats/REMAP Open Dataset PD/21h9f9e30v9cl2fapjggz4q1x7/Turning/Data/turning_2D3D_skeletons_coarsened/Turning_coarsen_CSV/Pt204_C_n_350/input_2D/keypoints.csv'
preprocessed_data = preprocess_new_data(filepath, scaler)
prediction = predict(model, preprocessed_data)
print(f'The predictions for the new data are: {prediction}')

The predictions for the new data are: ['Control']
